# Iterables

With our current setup we have graphs like this that analyses data from one subject

![](http://nipy.org/nipype/_images/graphviz-b65596b443fdc105a6508acf176afcc6261aa65a.png)

Let's assume that we want to run our analysis over different subjects. The resulting graph 
would look something like this:
![](http://nipy.org/nipype/_images/graphviz-675dc232294d6cecf3284efeace35923746f8ac8.png)


This can be performed with [iterables](http://nipy.org/nipype/users/mapnode_and_iterables.html#iterables)

In [ ]:
import os
try:
    os.chdir(notebook_path)
except:
    notebook_path = os.path.abspath('.')

output_path = os.path.abspath('outputs')
if not os.path.exists(output_path):
    os.mkdir(output_path)
    
wd_path = os.path.join(output_path, '06_iterables')
if not os.path.exists(wd_path):
    os.mkdir(wd_path)
os.chdir(wd_path)
print(wd_path)

In [ ]:
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces.io import DataSink, SelectFiles
from nipype.interfaces.nipy.preprocess import Trim, FmriRealign4d

workflow_path = os.path.join(wd_path, 'wf_4')
wf = Workflow(name='iterables_wf')
wf.base_dir = workflow_path
wf.config['execution']['crashdump_dir'] = os.path.join(workflow_path, 'crash')
wf.config['execution']['remove_unnecessary_outputs'] = False

Note that we now have an iterable to the input 'subject_id'.

This interable is a list of 2 strings ('s01', 's02').

In the rs string, '{subject_id}' is replaced by 's01' and 's02', respectively.

In [ ]:
data_path = os.path.join(notebook_path, 'data')

file_template = {'funct': 'ds107/{subject_id}/BOLD/task001_run001/bold.nii.gz'}

selectfiles = Node(SelectFiles(file_template, base_directory=data_path),
                   name="selectfiles")
selectfiles.iterables = ('subject_id', ['sub001', 'sub044'])

In [ ]:
trim = Node(Trim(), 'trim')
trim.inputs.end_index = 10
wf.connect(selectfiles, 'funct', trim, 'in_file')

realign = Node(FmriRealign4d(), 'realign')
realign.inputs.tr = 3.
realign.inputs.time_interp = True
realign.inputs.slice_order = range(0,35)

wf.connect(trim, 'out_file', realign, 'in_file')

In [ ]:
ds_dir = os.path.join(wd_path, 'output_subjects')
ds = Node(DataSink(), name='ds')
ds.inputs.base_directory = ds_dir
ds.inputs.regexp_substitutions = [
    ('_subject_id_', '')
]

In [ ]:
wf.connect(realign, 'out_file', ds, 'realigned_data')

In [ ]:
wf.run()

In [ ]:
!ls

In [ ]:
!ls output_subjects/realigned_data

In [ ]:
!ls output_subjects/realigned_data/sub044/